In [ ]:
!pip install pyscf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyscf
import numpy as np
from sympy import *
import math
from scipy.spatial import distance

In [ ]:
#diagonalize using symmetric orthogonalization
def diagonalize_vector(overlap):
    S_eigenvalues, U = np.linalg.eigh(overlap)

    # Diagonalize S: s = (U_transpose.S).U
    s = U.transpose().dot(overlap).dot(U)
    s_rounded = np.around(s, decimals = 5)
    s_sqrtinv = np.linalg.inv(np.sqrt(s_rounded))
    diagonal = U.dot(s_sqrtinv).dot(U.transpose())
    return diagonal

In [ ]:
# define our two electron part of Fock matrix (G)

def define_G(density, e2e, leng):
    G = np.zeros((leng,leng))
    for mu in range(leng):
        for nu in range(leng):
            set_val = 0
            for la in range(leng):
                for si in range(leng):
                    set_val += density[la][si] * (e2e[mu][nu][si][la] - 0.5 * e2e[mu][la][si][nu])

            G[mu][nu] = set_val
  
  return G

In [ ]:
def calc_F_prime(diagonal,F):

    F_pr= diagonal.transpose().dot(F).dot(diagonal)
    return F_pr


In [ ]:
#recalculate the new density matrix
def new_density(C1, leng, N):
    new_P = np.zeros((leng,leng))
    for mu in range(leng):
        for nu in range(leng):
            set_val = 0
            for la in range(int(N/2)):
                set_val += (2 * C1[mu][la]*C1[nu][la])

            new_P[mu][nu] = set_val
  
    return new_P

  

In [ ]:
#compare the old and new density matrices to determine the degree of difference
def delta_P(newP, oldP,leng):
    deviation = np.sum(np.power(newP-oldP,2))
    deviation = deviation / (leng*leng) #double check
    return math.sqrt(deviation)

In [ ]:
# extract charge of molecule
def get_Z(molecule):
    Z = []
    for i in (molecule._atm):
        Z.append(i[0])
    return Z

In [ ]:
#calculate the nuclear energies for non-pyscf integrals
def energy_nucl(r,Z):
    nuc_en = 0
    for i in range(len(r)):
        for j in range(i+1, len(r)):
            nuc_en += Z[i]*Z[j]/distance.euclidean(r[i],r[j])

    return nuc_en

#calculate the nuclear energies for pyscf integrals
def energy_nucl_pyscf(molecule):
    return molecule.energy_nuc()

In [ ]:
#calculate the electronic energy from the density matrix
def energy_elec(new_P, coreham, F,leng):
    elec_en = 0
    for mu in range(leng):
        for nu in range( leng):
            elec_en+= new_P[nu][mu]* (coreham[mu][nu]+F[mu][nu])
    return elec_en*0.5

In [ ]:
'''
SCF Procedure using Pyscf Integrals
'''
max_iter = 50
i=0

molecule = pyscf.gto.M(atom = 'H 0 0 0; H 1.463 0 0', basis = 'sto-3g')
rh2=[[0.0, 0.0, 0.0], [1.463, 0.0, 0.0]] #hardcode coordinates (x,y,z)
Z = get_Z(molecule)
molecule.build()

kE = molecule.intor('int1e_kin',aosym = 's1')
pE_nuc = molecule.intor('int1e_nuc', aosym = 's1')
overlap = molecule.intor('int1e_ovlp',aosym = 's1')
e2e = molecule.intor('int2e',aosym = 's1')
coreham = kE+pE_nuc

diagonal=diagonalize_vector(overlap)
guess_den_matrix = (np.ones((len(overlap[0]),len(overlap[0]))))
new_P=np.zeros([len(overlap[0]), len(overlap[0])])

while((delta_P(new_P, guess_den_matrix, len(overlap[0]))>0.0001) and (i < max_iter)):
    i+=1
  
    guess_den_matrix=new_P

    G=define_G(guess_den_matrix,e2e,len(overlap))
    F=coreham+G 

    F_pr= calc_F_prime(diagonal, F)
 
    eps,C_pr = np.linalg.eigh(F_pr)
  
    C1 = diagonal.dot(C_pr)
  
    N= molecule.nelectron

    new_P= new_density(C1, len(overlap[0]), N)
  
    print('deltaP: '+str(delta_P(new_P, guess_den_matrix, len(overlap[0])))+' for iteration: '+str(i))
if((delta_P(new_P, guess_den_matrix, len(overlap[0]))<=0.0001)):
    print('Converged after '+str(i)+' iterations to density matrix\n '+str(new_P))
else:
    print('Did not converge in '+ str(i)+' iterations')

print('\n')
nucenergy = energy_nucl_pyscf(molecule)
print('Nuclear Energy: '+str(nucenergy))
elecenergy = energy_elec(new_P, coreham, F, len(overlap))
print('Electronic Energy: '+str(elecenergy))
totalE = nucenergy+elecenergy
print('Total Energy of System: '+str(totalE))

print('\n')
print('Energies calculated using the given pyscf HF functions (used for comparison)')
mf = pyscf.scf.RHF(molecule)
mf.kernel()
print('Nuclear Energy: '+str(mf.energy_nuc()))
print('Electronic Energy: '+ str(mf.energy_elec()[0]))
print('Total Energy: ' + str(mf.energy_tot()))

deltaP: 0.7869801995781783 for iteration: 1
deltaP: 0.0 for iteration: 2
Converged after 2 iterations to density matrix
 [[0.7869802 0.7869802]
 [0.7869802 0.7869802]]


Nuclear Energy: 0.3617069110868079
Electronic Energy: -1.283753178330831
Total Energy of System: -0.922046267244023


Energies calculated using the given pyscf HF functions (used for comparison)
converged SCF energy = -0.922046885373691
Nuclear Energy: 0.3617069110868079
Electronic Energy: -1.2837537964604993
Total Energy: -0.9220468853736914


In [ ]:
'''
SCF Procedure using example Hartree Fock integrals which were calculated in the sample code
'''
max_iter = 50
i=0

molecule = pyscf.gto.M(atom = 'H 0 0 0; H 1.463 0 0', basis = 'sto-3g')
rh2=[[0.0, 0.0, 0.0], [1.463, 0.0, 0.0]] #hardcode coordinates (x,y,z)
Z = get_Z(molecule)
molecule.build()

#Integrals from HF Sample Code
e2e = np.array([[[[0.7746083600328786, 0.31179457036897384], [0.3117945703689739, 0.6057033663335998]], [[0.31179457036897396, 0.17726712195066144], [0.17726712195066147, 0.43727932526541696]]], [[[0.3117945703689739, 0.17726712195066147], [0.17726712195066147, 0.43727932526541696]], [[0.6057033663335994, 0.43727932526541685], [0.43727932526541696, 1.3071516075554823]]]])
overlap= np.array([[1.0000014259978642, 0.4507704116477876],[0.45077041164778753, 1.0000014259978642]])
coreham= np.array([[-1.09920546, -0.79574889],[-0.79574889, -0.58283112]])

diagonal=diagonalize_vector(overlap)
guess_den_matrix = (np.ones((len(overlap[0]),len(overlap[0]))))
new_P=np.zeros([len(overlap[0]), len(overlap[0])])

while((delta_P(new_P, guess_den_matrix, len(overlap[0]))>0.0001) and (i < max_iter)):
    i+=1
    
    guess_den_matrix=new_P

    G=define_G(guess_den_matrix,e2e,len(overlap))
    F=coreham+G 

    F_pr= calc_F_prime(diagonal, F)
 
    eps,C_pr = np.linalg.eigh(F_pr)
  
    C1 = diagonal.dot(C_pr)
  
    N= molecule.nelectron

    new_P= new_density(C1, len(overlap[0]), N)
  
    print('deltaP: '+str(delta_P(new_P, guess_den_matrix, len(overlap[0])))+' for iteration: '+str(i))
if((delta_P(new_P, guess_den_matrix, len(overlap[0]))<=0.0001)):
    print('Converged after '+str(i)+' iterations to density matrix\n '+str(new_P))
else:
    print('Did not converge in '+ str(i)+' iterations')

print('\n')
nucenergy = energy_nucl(rh2, Z)
print('Nuclear Energy: '+str(nucenergy))
elecenergy = energy_elec(new_P, coreham, F, len(overlap))
print('Electronic Energy: '+str(elecenergy))
totalE = nucenergy+elecenergy
print('Total Energy of System: '+str(totalE))

deltaP: 0.7599773480431314 for iteration: 1
deltaP: 0.019017491090931413 for iteration: 2
deltaP: 0.0012684805826594812 for iteration: 3
deltaP: 8.667478219283301e-05 for iteration: 4
Converged after 4 iterations to density matrix
 [[1.33262637 0.51741981]
 [0.51741981 0.20089897]]


Nuclear Energy: 0.683526999316473
Electronic Energy: -1.6593120239447776
Total Energy of System: -0.9757850246283046
